socket编程

In [ ]:
import socket

# tcp
sk_tcp = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
#upd
sk_udp = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)


sk_tcp.close()

sk_udp.close()

关于线程的使用

In [16]:
import threading


def say_sth(sth,num):
    print(sth)
    print(num)
    
    
#创建线程并启动
t = threading.Thread(target=say_sth,args=([1,2,3],10))
t.start()

# #获取当前(进程)所有线程
# print(threading.enumerate())



class YourClass(threading.Thread):
    #继承Thread类必须实现run方法，才能通过start方法来启动
    def run(self):
        print("do your job")


        
# t = YourClass()
# t.start()


[1, 2, 3]
10
